In [1]:
from numpy import linalg as LA
import torch, foolbox, random
import numpy as np
from models import MNIST, CIFAR10, IMAGENET, SimpleMNIST, load_mnist_data, load_cifar10_data, imagenettest, load_model
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [2]:
train_loader, test_loader, train_dataset, test_dataset = load_cifar10_data()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = CIFAR10()
net.cuda()
net = torch.nn.DataParallel(net, device_ids=[0])
load_model(net, 'models/cifar10_gpu.pt')
net.eval()
model = net.module if torch.cuda.is_available() else net

In [4]:
fmodel = foolbox.models.PyTorchModel(model, bounds=[0,1], num_classes=10)
attack = foolbox.attacks.BoundaryAttack(fmodel)

In [ ]:
random.seed(0)
avg_dis = 0
avg_calls = 0
num_imgs = 1
for _ in range(1):
    idx = random.randint(100, len(test_dataset)-1)
    print(idx)
    image, label = test_dataset[idx]
    image = image.numpy()
    targets = list(range(10))
    targets.pop(label)
    target = random.choice(targets)
#for i in range(50):101
#    img, label2 = test_dataset[i]
#    if label2 == 6:
#        target = img.numpy()
    adv = attack(image,label,iterations=5000, verbose=False, unpack=False, log_every_n_steps=1000)
    avg_dis += LA.norm(adv.image-image)
    avg_calls += adv._total_prediction_calls

print(avg_dis/num_imgs, avg_calls/num_imgs)


6411
run with verbose=True to see details
Step 0: 3.01750e-02, stepsizes = 1.0e-02/1.0e-02: 
Step 1000: 3.32122e-05, stepsizes = 3.4e-02/5.9e-04:  (took 0.01120 seconds)
Step 2000: 1.70608e-05, stepsizes = 1.5e-02/1.7e-04: d. reduced by 0.03% (5.9198e-09) (took 0.01564 seconds)
Step 3000: 1.36708e-05, stepsizes = 1.0e-02/1.2e-04:  (took 0.01070 seconds)


In [8]:
adv = attack(image,label,iterations=5000, verbose=True, unpack=False, threaded_gen=1, threaded_rnd=1)

Neither starting_point nor initialization_attack given. Falling back to BlendedUniformNoiseAttack for initialization.
Initial spherical_step = 0.01, source_step = 0.01
Using 1 threads to create random numbers
Step 0: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02: 
Step 1: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11750 seconds)
Step 2: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.10934 seconds)
Step 3: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11061 seconds)
Step 4: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11045 seconds)
Step 5: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.10956 seconds)
Step 6: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11135 seconds)
Step 7: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11029 seconds)
Step 8: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11083 seconds)
Step 9: 8.13248e-03, stepsizes = 1.0e-02/1.0e-02:  (took 0.11176 seconds)
Step 10: 7.97064e-03, stepsizes = 1.0e-02/1.0e-02: d. reduced by 1.99% (1.

KeyboardInterrupt: 

In [18]:
from numpy import linalg as LA
LA.norm(adv.image-image)

0.18515235

In [19]:
adv._total_prediction_calls

111806

In [9]:
adv._crit

AttributeError: 'Adversarial' object has no attribute '_crit'